In [3]:
import torch
import copy
import numpy as np
import itertools
import random as rd

# Purpose

Purpose of this branch is to test convergence properties of the network using one-hot-vectors for the state space instead of 0-3, as the OHV format may be easier for the network to operate on. Due to the need to operate on zero vectors we will

- add a small noise function gamma to each OHV
- use He initialisation for weight initialisation

In [17]:
class Environment():
    
    #fields:
    #a grid of size defined by parameter
    #b, a tuple repsenting the delivery location
    #a, a tuple representing the package location
    #size, an int representing the horizontal and vertical dimensions of the grid
    def __init__(self, size:int):
        self.size = size
        self.actions = ['north', 'south', 'west', 'east']
        self.reward_pickup = 10
        self.reward_deliver = 10
        self.reward_move = -0.1
        self.reset_env()
    
    def reset_env(self):
        combinations = list(itertools.product(list(range(self.size)), repeat=2))
        rd.shuffle(combinations)
        self.a, self.b, self.agent_coords = combinations[0:3]
        self.collected = 0
        self.done = False
        self.test_length = 0 #an additional parameter to determine how long the environment has been active for
        self.setup_grid()

    def coordinates_to_noisy_ohv(self, coordinate:int):
        ohv = np.zeros(4)
        noise = np.random.normal(-1e-5, 1e-5, ohv.shape)
        ohv[coordinate] = 1
        ohv = ohv + noise
        return ohv

    #method to set up the original grid including a location
    def setup_grid(self):
        grid = np.zeros((self.size, self.size))
        self.grid = grid
        grid[self.a] = 1
        grid[self.b] = 2
        grid[self.agent_coords] = -1

    def get_state(self):
        #This method unpacks the coordinates into a tuple which can be used to index the qmatrix or for easy incorporation to the Deep Q Network
        coords = [self.agent_coords[0], self.agent_coords[1], self.a[0], self.a[1], self.b[0], self.b[1]]
        coords = [self.coordinates_to_noisy_ohv(c) for c in coords]
        coords.append(np.array(self.collected))
        return np.hstack(coords)
        return np.ndarray.flatten(np.array(coords))

        return np.array([self.agent_coords[0], self.agent_coords[1], self.collected, self.a[0], self.a[1], self.b[0], self.b[1]])
    
    #Method which updates the location of the agent on the grid. Currently just zeroes whatever it landed on - can include other logic instead
    def move_agent(self, action):
        #For agent move, note that 0 = up, 1=down, 2=left, 3=right
        self.test_length += 1

        #Assign new coordinates for agent to exist at
        new_x, new_y = self.agent_coords
        if action == 'north' and self.agent_coords[1] > 0:
            new_y -= 1
        elif action == 'south' and self.agent_coords[1] < self.size - 1:
            new_y += 1
        elif action == 'west' and self.agent_coords[0] > 0:
            new_x -= 1
        elif action == 'east' and self.agent_coords[0] < self.size - 1:
             new_x += 1
        
        #Update the grid based on new agent coordinates
        self.grid[self.agent_coords] = 0
        self.agent_coords = (new_x, new_y)
        self.grid[self.agent_coords] = -1

        #Handle logic based on new agent location
        if self.collected == 0:
            if self.agent_coords == self.a:
                self.collected = 1
                reward = self.reward_pickup
            else:
                reward = self.reward_move
        
        else:
            if self.agent_coords == self.b:
                self.done = True
                reward = self.reward_deliver
            else:
                reward = self.reward_move

                #This code was considered test code which could be useful in exiting early
        if self.test_length > 50:
            self.done = True
        return reward, self.done

    def get_grid(self):
        return self.grid.tolist()

    # def plot_grid(self, snapshot, ax=None):
    #     if ax is None:
    #         fig, ax = plt.subplots()
    #         ax.set_facecolor('white')
    #     else:
    #         ax.clear()
        
    #     # Plot the grid
    #     ax.imshow(np.array([[0]]), cmap="bone", extent=[0, self.size, 0, self.size])

    #     for i in range(self.size):
    #         for j in range(self.size):
    #             cell_value = snapshot[i][j]
    #             if cell_value == -1:
    #                 # Display agent image in the cell
    #                 imagebox = OffsetImage(agent_img, zoom=0.08)
    #                 ab = AnnotationBbox(imagebox, (j + 0.5, self.size - i - 0.5), frameon=False)
    #                 ax.add_artist(ab)
    #             elif cell_value == 1:
    #                 # Display package image in the cell
    #                 imagebox = OffsetImage(package_img, zoom=0.03)
    #                 ab = AnnotationBbox(imagebox, (j + 0.5, self.size - i - 0.5), frameon=False)
    #                 ax.add_artist(ab)
    #             elif cell_value == 2:
    #                 # Display destination image in the cell
    #                 imagebox = OffsetImage(destinationB_img, zoom=0.05)
    #                 ab = AnnotationBbox(imagebox, (j + 0.5, self.size - i - 0.5), frameon=False)
    #                 ax.add_artist(ab)
    #             else:
    #                 ax.text(j + 0.5, self.size - i - 0.5, self.grid[i, j], ha='center', va='center', fontsize=20, color='black')
        
    #     # Set axis properties
    #     ax.set_xlim(0, self.size)
    #     ax.set_ylim(0, self.size)
    #     ax.set_xticks(np.arange(self.size) + 1)
    #     ax.set_yticks(np.arange(self.size) + 1)
    #     ax.set_xticklabels([])
    #     ax.set_yticklabels([])
    #     ax.grid(True, linewidth=2, color='white')
        
    #     # Set title
    #     ax.set_title("Package Delivery Agent")
        
    #     # Show the plot
    #     #plt.show()
    #     return ax

In [9]:
# Define the TransportAgent agent
#new statespace size should be 25 - this represents six 4-size one-hot-vectors, plus a collected flag
class TransportAgent:
    def __init__(self, statespace_size=25, gamma=0.99, learning_rate=0.997, start_epsilon=1.0,
                 epsilon_decay_factor=0.997, min_epsilon=0.1, replay_buffer_size=1000,
                 batch_size=200, network_copy_frequency=500):
        self.statespace_size = statespace_size
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.model2 = self.prepare_torch()
        self.memory = []  # Using a list instead of a deque
        self.epsilon = start_epsilon
        self.epsilon_min = min_epsilon
        self.epsilon_decay_factor = epsilon_decay_factor
        self.replay_buffer_size = replay_buffer_size
        self.batch_size = batch_size
        self.network_copy_frequency = network_copy_frequency
        self.steps_since_copy = 0  # Counter for network copy

    def prepare_torch(self):
        l1 = self.statespace_size
        l2 = 150
        l3 = 100
        l4 = 4
        self.model = torch.nn.Sequential(
            torch.nn.Linear(l1, l2),
            torch.nn.ReLU(),
            torch.nn.Linear(l2, l3),
            torch.nn.ReLU(),
            torch.nn.Linear(l3, l4)
        )
        model2 = copy.deepcopy(self.model)
        model2.load_state_dict(self.model.state_dict())
        self.loss_fn = torch.nn.HuberLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        return model2

    def update_target(self):
        if self.steps_since_copy >= self.network_copy_frequency:
            self.model2.load_state_dict(self.model.state_dict())
            self.steps_since_copy = 0

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        if len(self.memory) > self.replay_buffer_size:
            self.memory.pop(0)  # Remove the oldest experience

    def get_qvals(self, state):
        state1 = torch.from_numpy(state).float()
        qvals_torch = self.model(state1)
        qvals = qvals_torch.data.numpy()
        return qvals

    def get_maxQ(self, s):
        return torch.max(self.model2(torch.from_numpy(s).float())).float()

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(range(4))  # Random action
        q_values = self.get_qvals(state)
        return np.argmax(q_values[0])  # Greedy action

    def process_minibatch(self, minibatch):
        states = []
        actions = []
        targets = []
        for state, action, reward, next_state, done in minibatch:
            q_values = self.get_qvals(state)
            if done:
                q_values[action] = reward
            else:
                q_values[action] = reward + self.gamma * self.get_maxQ(next_state)
            states.append(state)
            actions.append(action)
            targets.append(q_values[0])
        return np.array(states), np.array(actions), np.array(targets) #this is returning a thruple of state transitions?

    def train_one_step(self, states, actions, targets):
        state1_batch = torch.from_numpy(states).float()
        # state1_batch = torch.Tensor([torch.from_numpy(s).float() for s in states])
        # print(f"Somehow this is {state1_batch.shape}")
        # print(f"Shape of top {state1_batch[0].shape}")
        # print(f"shape of states: {states.shape}")
        action_batch = torch.Tensor(actions)
        # print(f"and actions have shape {action_batch.shape}")
        # print(f"while one action has shape{action_batch_1.shape}")
        Q1 = self.model(state1_batch)
        X = Q1.gather(dim=1, index=action_batch.long().unsqueeze(dim=1)).squeeze()
        Y = torch.tensor(targets)
        loss = torch.nn.MSELoss()(X, Y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()

In [18]:
if __name__ == "__main__":
    # env = TransportEnv()  # Create the environment
    env = Environment(4)
    agent = TransportAgent(learning_rate=0.997, start_epsilon=1.0, epsilon_decay_factor=0.99999,
                           min_epsilon=0.1, replay_buffer_size=500, batch_size=200, network_copy_frequency=100)

    episodes = 1000
    for episode in range(episodes):
        env.reset_env()
        state = env.get_state()  # Reset the environment and get the initial state
        done = False
        total_reward = 0
        while not done:
            # print("state", state)
            action = agent.act(state)
            reward, done = env.move_agent(env.actions[action])
            next_state = env.get_state()
            # next_state, reward, done = env.step(env.actions[action])
            total_reward += reward
            agent.remember(state, action, reward, next_state, done)
            # print("agent.memory[-1]",agent.memory[-1])
            state = next_state

            # Training
            if len(agent.memory) > agent.batch_size:
                minibatch_indices = np.random.choice(len(agent.memory), agent.batch_size, replace=False)
                minibatch = [agent.memory[i] for i in minibatch_indices]
                states_batch, actions_batch, targets_batch = agent.process_minibatch(minibatch)
                agent.train_one_step(states_batch, actions_batch, targets_batch)
                agent.epsilon = max(agent.epsilon * agent.epsilon_decay_factor, agent.epsilon_min)
            agent.steps_since_copy += 1
            agent.update_target()
        # print("state", state)
        print(f"Episode: {episode + 1}, Total Reward: {total_reward}, Epsilon: {agent.epsilon}")

Episode: 1, Total Reward: 5.000000000000016, Epsilon: 1.0
Episode: 2, Total Reward: 19.5, Epsilon: 1.0
Episode: 3, Total Reward: 5.000000000000018, Epsilon: 1.0
Episode: 4, Total Reward: -5.099999999999998, Epsilon: 1.0
Episode: 5, Total Reward: 5.000000000000006, Epsilon: 0.9998900054998355
Episode: 6, Total Reward: 16.2, Epsilon: 0.9994901274791796
Episode: 7, Total Reward: 5.000000000000012, Epsilon: 0.998980514928347
Episode: 8, Total Reward: 16.90000000000001, Epsilon: 0.9986509040991433
Episode: 9, Total Reward: 5.000000000000016, Epsilon: 0.9981417194452505
Episode: 10, Total Reward: 18.299999999999997, Epsilon: 0.997952089585813
Episode: 11, Total Reward: 19.5, Epsilon: 0.9978822350352069
Episode: 12, Total Reward: 5.0000000000000036, Epsilon: 0.9973734423045478
Episode: 13, Total Reward: -5.099999999999998, Epsilon: 0.9968649089933213
Episode: 14, Total Reward: 5.000000000000006, Epsilon: 0.996356634969256
Episode: 15, Total Reward: 16.800000000000004, Epsilon: 0.9960179296030

KeyboardInterrupt: 